In [1]:
import json
import os
import time
import requests

In [18]:
import psycopg2
import os
import json

database_dict = {
        "database": os.environ.get("POSTGRES_DB"),
        "user": os.environ.get("POSTGRES_USERNAME"),
        "password": os.environ.get("POSTGRES_PASSWORD"),
        "host": os.environ.get("POSTGRES_WRITER"),
        "port": os.environ.get("POSTGRES_PORT"),
    }
engine = psycopg2.connect(**database_dict)
cur = engine.cursor()

def get_mainnet_daos(cur, engine):
    execute_string = f"SELECT * FROM daohaus.dao"
    cur.execute(execute_string)
    records = cur.fetchall()
    js = [];
    for x in records:
        if(x[5] == 1):
            curl = []
            curl.append(x[0])
            curl.append(x[1])
            js.append(curl)
    return js

def get_all_members(cur, engine):
    execute_string = f"SELECT * FROM daohaus.member"
    cur.execute(execute_string)
    records = cur.fetchall()
    js = [];
    for x in records:
        curl = []
        curl.append(x[1])
        curl.append(x[6])
        js.append(curl)
    return js

main_net_daos = get_mainnet_daos(cur, engine)
all_members = get_all_members(cur, engine)

In [35]:
dict_of_main_net_daos = {}
for dao in main_net_daos:
    dict_of_main_net_daos[dao[0]] = dao[1]
main_net_members = []
for member in all_members:
    if member[0] in dict_of_main_net_daos:
        curl = []
        curl.append(dict_of_main_net_daos[member[0]])
        curl.append(member[1])
        main_net_members.append(curl)

In [36]:
main_net_members

[['...'],
 ...]

In [3]:
API_KEY = "ASKMW73735RNXSZCUDNZPFU1W2URIWYXRE"

In [4]:
addresses = main_net_daos

In [5]:
all_tokens = set()
list_of_sets_tokens_in_each_address = []

In [6]:
counter = 0
for addy in addresses:
    address = str(addy[1])
    cur_add = address
    url = f"https://api.etherscan.io/api?module=account&action=tokentx&address={address}&page=1&offset=100&startblock=0&endblock=27025780&sort=asc&apikey={API_KEY}"
    response = requests.get(url)
    counter+=1
    if(response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        test_data = response.json()
    else:
        break
        
        
    cd = dict()
    current_address_set_of_tokens = set()
    for transaction in test_data['result']:
        all_tokens.add(transaction['contractAddress'])
        current_address_set_of_tokens.add(transaction['contractAddress'])
        
    cd[str(cur_add)] = current_address_set_of_tokens
    list_of_sets_tokens_in_each_address.append(cd)
    if counter % 20 == 0:
        print("SLEPT")
        time.sleep(1)

SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT
SLEPT


In [7]:
all_tokens_with_spot_prices = {}
for token in all_tokens:
    all_tokens_with_spot_prices[str(token)] = 0
    
did_not_work_tokens = []

In [8]:
counter = 0
for token in all_tokens:
    fl = True
    response = requests.get(f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={token}&apikey={API_KEY}")
    if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        time.sleep(1)
        response = requests.get(f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={token}&apikey={API_KEY}")
        if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
            time.sleep(1)
            response = requests.get(f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={token}&apikey={API_KEY}")
            if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
                did_not_work_tokens.append(token)
                fl = False
            else:
                test_data = response.json()
        else:
            test_data = response.json()
    else:
        test_data = response.json()
        
    if fl and test_data['message'] == "OK":
        if(len(test_data['result'])> 0):
            cd = {"tokenPriceUSD": test_data['result'][0]['tokenPriceUSD'], "tokenName" : test_data['result'][0]['tokenName'], "divisor":test_data['result'][0]['divisor'], "symbol" : test_data['result'][0]['symbol']}
            print(test_data['result'][0]['tokenPriceUSD'],test_data['result'][0]['tokenName'],test_data['result'][0]['divisor'])
            all_tokens_with_spot_prices[str(token)] = cd
        else:
            did_not_work_tokens.append(token)
        
    counter+=1
    if(counter % 2 == 1):
        time.sleep(1.5)

0.000000000000000000 Katerium.com 18
1.000000000000000000 Tether USD 6
1.000000000000000000 Sai Stablecoin v1.0 18
13.690000000000000000 Uniswap 18
1512.790000000000000000 ETH RSI 60/40 Yield II 18
0.000000000000000000 Lux Expression 18
1.240000000000000000 Chai 18
0.000000000000000000 CyberFM 18
0.997772000000000000 Dai Stablecoin 18
0.000000000000000000 a!NEVER VISIT www.168pools.com to check DeFi ROi ! 0
0.000000000000000000 Game Mine Alliance Community 18
27971.000000000000000000 yearn.finance 18
0.069907000000000000 BITTO 18
8.720368254700000000 Index 18
0.000000000000000000 DAOSquare 18
0.997605000000000000 USD Coin 6
0.000000079892000000 CyberFM Radio 18
0.000000000000000000 A! WWW.SPACESWAP.APP ! TOP DEFI AGGREGATOR ! 0
0.000000000000000000 Metta Token 2
2731.100000000000000000 Wrapped Ether 18
18.760000000000000000 ChainLink Token 18
0.000000000000000000 Huh 18
48.330000000000000000 MetaFactory 18
0.000000000000000000 DSCVR 18
0.000000000000000000 Fraktal 18
0.0000000000000000

In [9]:
js_str = json.dumps(all_tokens_with_spot_prices)
js_fin = json.loads(js_str)
with open('./list_of_all_mainnet_tokens_with_spot_prices_for_daos.json', 'w') as f:
        json.dump(js_fin, f)

In [12]:
md_of_dao_data_with_balances_of_each_token = {}

In [13]:
counter = 0
for member_dict in list_of_sets_tokens_in_each_address:
    list_of_all_tokens_with_balances_for_current_member = []
    for address in member_dict:
        for token in member_dict[address]:
            if(counter%20 == 0):
                time.sleep(1)
            cd = {}
            response = requests.get(f"https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress={token}&address={address}&tag=latest&apikey={API_KEY}")
            counter += 1
            if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
                response = requests.get(f"https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress={token}&address={address}&tag=latest&apikey={API_KEY}")
                counter+=1
                test_data = response.json()
            else:
                test_data = response.json()
            cd["contract_address"] = token
            cd["balance"] = test_data['result']
            print(address, token, test_data["result"])
            list_of_all_tokens_with_balances_for_current_member.append(cd)
        
        md_of_dao_data_with_balances_of_each_token[address] = list_of_all_tokens_with_balances_for_current_member

0x0557b380dd0f96d04829cfe7f8440d2b61bdb345 0x6b175474e89094c44da98b954eedeac495271d0f 0
0x016e79e9101a8eaa3e7f46d6d1c267819c09c939 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0
0x016e79e9101a8eaa3e7f46d6d1c267819c09c939 0xd15ecdcf5ea68e3995b2d0527a0ae0a3258302f8 109103000037330037082820
0x098eabc5cb8ca7d8aaeaf0a927a5c2bbcedffb30 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 100000000000000000
0x0042dfb8301607df72ec9fdab39759371f827bb0 0x6b175474e89094c44da98b954eedeac495271d0f 0
0x0372f3696fa7dc99801f435fd6737e57818239f2 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0
0x108e0ac5241dd65f1213396681a799ed049dd5dc 0x6b175474e89094c44da98b954eedeac495271d0f 0
0x1621e5d7b9bb0e1330897f4f5df1fba9eb399263 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 30000000000000000
0x1bbab7783d2dbe69f636cfa182dbc67e3f6e5093 0x6b175474e89094c44da98b954eedeac495271d0f 0
0x1b3d7efb93ec432b0d1d56880c23303979b379e9 0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359 10000000000000000000
0x1b3d7efb93ec432b0d1d56880c23303979b379e9 0x

In [14]:
all_wallet_totals = {}
final_data_set = {}

In [15]:
for address in md_of_dao_data_with_balances_of_each_token:
    total = 0
    for token in md_of_dao_data_with_balances_of_each_token[address]:
        if(len(md_of_dao_data_with_balances_of_each_token[address]) > 0):
            if type(all_tokens_with_spot_prices[token['contract_address']]) is dict:
                price = float(all_tokens_with_spot_prices[token['contract_address']]["tokenPriceUSD"])
                divisor = int(all_tokens_with_spot_prices[token['contract_address']]["divisor"])
                balance = int(token['balance'])
                divided_balance = balance/ (10**divisor)
                value = divided_balance * price
                total+=value
    
    
    all_wallet_totals[address] = total

In [16]:
ml_of_all_addresses_and_token_info = []

In [ ]:
for address in md_of_dao_data_with_balances_of_each_token:
    address_final_dict = {}
    address_final_dict["address"] = address

    enriched_list_of_tokens = []
    for token in md_of_dao_data_with_balances_of_each_token[address]:
        if type(all_tokens_with_spot_prices[token['contract_address']]) is dict:
            cd = {}
            cd['contract_address'] = token['contract_address']
            cd['token_name'] = all_tokens_with_spot_prices[token['contract_address']]["tokenName"]
            cd['divisor'] = all_tokens_with_spot_prices[token['contract_address']]["divisor"]
            cd['symbol'] = all_tokens_with_spot_prices[token['contract_address']]["symbol"]
            cd['balance'] = token['balance']
            enriched_list_of_tokens.append(cd)

    address_final_dict["token_info"] = enriched_list_of_tokens
    address_final_dict["wallet_total"] = all_wallet_totals[address]
    address_final_dict["members"] = []
    ml_of_all_addresses_and_token_info.append(address_final_dict)

In [8]:
for dao in ml_of_all_addresses_and_token_info:
    dao['members'] = []

In [37]:
for member in main_net_members:
    for dao in ml_of_all_addresses_and_token_info:
        if(dao['address'] == member[0]):
            dao['members'].append(member[1])

In [18]:
js_str = json.dumps(ml_of_all_addresses_and_token_info)
js_fin = json.loads(js_str)
with open('./all_mainnet_dao_data_with_token_balances_and_wallet_totals.json', 'w') as f:
        json.dump(js_fin, f)